# ``super()`` behavior:

What gets called?, and when does it get called?

or:

### What does super() really do?

## The three "rules" of super():

Raymond Hettinger's rules for ``super()``

1. The method being called by super() needs to exist.

2. The caller and callee need to have a matching argument signature.

3. Every occurrence of the method needs to use super()

And a key point:

Using `super()` is *part* of the interface of the class. Anything that subclasses from your class needs to know that super was used.

In [1]:
class A():
    def __init__(self):
        print("in A __init__")
        print("self's class is:", self.__class__)
        s = super().__init__()

class B():
    def __init__(self):
        print("in B.__init__")
        print("self's class is:", self.__class__)
        s = super().__init__()

class C():
    def __init__(self):
        print("in C.__init__")
        print("self's class is:", self.__class__)
        s = super().__init__()

class D(C, B, A):
    def __init__(self):
        print("in D.__init__")
        print("self's class is:", self.__class__)
        super().__init__()
        

In [2]:
# take a look at D's method resolution order:
print("D's mro:")
for c in D.__mro__:
    print(c)

D's mro:
<class '__main__.D'>
<class '__main__.C'>
<class '__main__.B'>
<class '__main__.A'>
<class 'object'>


In [3]:
# What happens when you initilize a D object
d = D()

in D.__init__
self's class is: <class '__main__.D'>
in C.__init__
self's class is: <class '__main__.D'>
in B.__init__
self's class is: <class '__main__.D'>
in A __init__
self's class is: <class '__main__.D'>


## super's parameters

To do its thing, super() needs to know two things:

1) It needs to know that type (class) that you want the super-classes of

2) It needs to know the actual object instance at the time it is called.

In fact, in Python 2, you had to explicitly tell super what these were:

``super(type, obj)``

python3 fills these in for you at run time, but in python2, you needed to specify them, calling it like so:

```
class A(object):
    def __init__(self):
        super(A, self).__init__()
```

But why do you need BOTH `A` and `self`? -- isn't `self` an instance of `A`?

Not neccesarily -- if A's method is being called from a subclass, then `self` will be an instance of the subclass. `super()` requires that the object be an instance of the class (or a subclass).

This distiction will come up later....

Again, py3 takes care of this for you, though you CAN still spell it out.

In [4]:
super?

In [5]:
# you can call super outside of a class definiton is you pass in the
# type and instance:

s_c = super(C, d)
print(s_c)

<super: <class 'C'>, <D object>>


This works because `d` is a `D` object, which is a subclass of `C`.

In [6]:
c = C()

in C.__init__
self's class is: <class '__main__.C'>


In [7]:
# try an invalid relationship:
super(D, c)


TypeError: super(type, obj): obj must be an instance or subtype of type

This fails: `C` is NOT a subclass of `D`

In [8]:
s_a = super(A, d)
print(s_a)

<super: <class 'A'>, <D object>>


In [9]:
s_b = super(B, d)
print(s_b)

<super: <class 'B'>, <D object>>


In [10]:
print(D.__mro__)

(<class '__main__.D'>, <class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <class 'object'>)


## Rule 3: Use super() everywhere.

This is key: if you want to use super() in a subclass, it needs to be used in all the superclasses:

In [11]:
# without super() at all -- calling each subclass' method explicitly:

class A():
    def this(self):
        print("in A.this")

class B():
    def this(self):
        print("in B.this")

class C(A,B):
    def this(self):
        print("in C.this")
        A.this(self)
        B.this(self)

In [12]:
print("Running without super()")
c = C()
c.this()

Running without super()
in C.this
in A.this
in B.this


C's `this` explicitly called both A and B's methods -- so they all get called.

### Using super in just C:

Explicitly calling subclass methods gets tedious -- let's use super()

In [13]:
class A():
    def this(self):
        print("in A.this")

class B(A):
    def this(self):
        print("in B.this")

class C(B):
    def this(self):
        print("in C.this")
        super().this()

In [14]:
for c in C.__mro__:
    print(c)
c = C()
c.this()

<class '__main__.C'>
<class '__main__.B'>
<class '__main__.A'>
<class 'object'>
in C.this
in B.this


**Note:**  `A.this` did NOT get called!

Even though it is in in the MRO.

Python stopped when it found the method in B.

### Using super everywhere:

In [15]:
class Base():
    def this(self):
        pass # just so there is a base that has the method

class A(Base):
    def this(self):
        print("in A.this")
        super().this()

class B(Base):
    def this(self):
        print("in B.this")
        super().this()
class C(A,B):
    def this(self):
        print("in C.this")
        super().this()

In [16]:
c = C()
c.this()
print(Base.__mro__)

in C.this
in A.this
in B.this
(<class '__main__.Base'>, <class 'object'>)


Now both A and B's methods get called -- probably what you want.

But if you don't want both called -- better to just be Explicit, rather than use super():

In [17]:
class Base():
    def this(self):
        pass # just so there is a base that has the method

class A(Base):
    def this(self):
        print("in A.this")
        super().this()

class B(Base):
    def this(self):
        print("in B.this")
        super().this()

class C(A,B):
    def this(self):
        print("in C.this")
        A.this(self)

# note: not calling super() in C...

In [18]:
c = C()
c.this()

in C.this
in A.this
in B.this


**Whoa** -- A and B's method DID get called! -- why?

We only called's A's `this` method.

In [19]:
A.__mro__

(__main__.A, __main__.Base, object)

B is not there.

But:

In [20]:
class Base():
    def this(self):
        pass # just so there is a base that has the method

class A(Base):
    def this(self):
        print("in A.this")
        print("self's class:", self.__class__)
        super().this()

class B(Base):
    def this(self):
        print("in B.this")
        super().this()

class C(A,B):
    def this(self):
        print("in C.this")
        A.this(self)

In [21]:
c = C()
c.this()
print("C's MRO:")
for c in C.__mro__:
    print(c)

in C.this
in A.this
self's class: <class '__main__.C'>
in B.this
C's MRO:
<class '__main__.C'>
<class '__main__.A'>
<class '__main__.B'>
<class '__main__.Base'>
<class 'object'>


Remember, `super()` is dynamic -- what it calls is determined at run time.

That means A's `this` method knows that it's a C object when super is called. And C has B in it's MRO.

That's how it knows to call ``B``'s method too. 

## Use `super` everywhere, or not at all

If you don't want all the superclass' methods called, and want to be explicit, you need to not use `super` at all.

In [22]:
class Base():
    def this(self):
        pass # just so there is a base that has the method

class A(Base):
    def this(self):
        print("in A.this")
        print("self's class:", self.__class__)

class B(Base):
    def this(self):
        print("in B.this")

class C(A,B):
    def this(self):
        print("in C.this")
        A.this(self)

In [23]:
c = C()
c.this()

in C.this
in A.this
self's class: <class '__main__.C'>


Without super, only the methods you specifically call get used.

This is why we say that using `super()` is *part* of the interface of the class.
